<a href="https://colab.research.google.com/github/punch-boxing/punch-ml/blob/main/punch-ml-torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Settings

In [70]:
# default values
PUNCH_TYPES = {
    "None": 0,
    "Straight": 1,
    "Hook": 2,
    "Body": 3,
    "Uppercut": 4,
}

# values for model training
INPUT_SIZE = 3
HIDDEN_SIZE = 10
NUM_LAYERS = 2
WINDOW_SIZE = 20


# values for preparing data
MAX_DATA_NUMBER = 15 # file counts
ACCELERATION_DATA_TYPE = "Acceleration" # "Acceleration" for "Raw Acceleration"

# 1. Preparing Model

In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PUNCH_ML(torch.nn.Module):
    def __init__(self):
        super(PUNCH_ML, self).__init__()

        # a layer that puts the accelerometer value into tanh function, this layer makes the accelerometer value from -1 ~ 1
        self.tanh = nn.Tanh()

        self.gru = nn.GRU(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS)
        self.linear = nn.Linear(HIDDEN_SIZE, len(PUNCH_TYPES))

    def forward(self, x):
        x = self.tanh(x)
        x, _ = self.gru(x)
        x = self.linear(x[-1])
        return x


# 2. Preparing Data

In [72]:
import pandas as pd

raw_datas = []

# Columns
# Index,Time,Raw Acceleration X,Raw Acceleration Y,Raw Acceleration Z,Acceleration X,Acceleration Y,Acceleration Z,Angular Velocity X,Angular Velocity Y,Angular Velocity Z,Orientation X,Orientation Y,Orientation Z,Punch Type

for i in range(1, MAX_DATA_NUMBER + 1):
  try:
    data = pd.read_csv(f"{i}.csv")
    data["Punch Index"] = data["Punch Type"].map(PUNCH_TYPES).fillna(0).astype(int)
    raw_datas.append(data)
  except:
    print(f"No data for {i}.csv")

No data for 2.csv
No data for 3.csv
No data for 4.csv
No data for 5.csv
No data for 6.csv
No data for 7.csv
No data for 8.csv
No data for 9.csv
No data for 10.csv
No data for 11.csv
No data for 12.csv
No data for 13.csv
No data for 14.csv
No data for 15.csv


In [79]:
import numpy as np
from sklearn.model_selection import train_test_split

datas = []
targets = []

for data in raw_datas:
  x = data[ACCELERATION_DATA_TYPE + " X"].values
  y = data[ACCELERATION_DATA_TYPE + " Y"].values
  z = data[ACCELERATION_DATA_TYPE + " Z"].values
  punch = data['Punch Index'].values

  for i in range(len(data) - WINDOW_SIZE):
    datas.append(np.array([
        x[i:i+WINDOW_SIZE],
        y[i:i+WINDOW_SIZE],
        z[i:i+WINDOW_SIZE],
    ]).T) # Transpose to get shape (WINDOW_SIZE, num_features)
    targets.append(punch[i + WINDOW_SIZE])


X = np.array(datas)
y = np.array(targets)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train)),
    batch_size=64,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test)),
    batch_size=64,
    shuffle=False
)

for data, target in train_loader:
  print(data.shape, target.shape)
  break

torch.Size([64, 20, 3]) torch.Size([64])


# 3. Compiling Model

In [96]:
from torchsummary import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using Device : {device}")

model = PUNCH_ML().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

summary(model, (WINDOW_SIZE, INPUT_SIZE))


Using Device : cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
              Tanh-1                [-1, 20, 3]               0
               GRU-2  [[-1, 20, 10], [-1, 20, 10]]               0
            Linear-3                    [-1, 5]              55
Total params: 55
Trainable params: 55
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.30
Params size (MB): 0.00
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


# 4. Training Model

In [83]:
best_loss = float('inf')
epochs = 100
patience = 5
counter = 0

for epoch in range(epochs):
  model.train()
  train_loss = .0

  for data, target in train_loader:
    data, target = data.permute(1, 0, 2).to(device, dtype = torch.float32), target.to(device)
    output = model(data)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  train_loss /= len(train_loader)

  model.eval()
  val_loss = .0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.permute(1, 0, 2).to(device, dtype = torch.float32), target.to(device) # Added dtype=torch.float32 here
      output = model(data)
      loss = criterion(output, target)
      val_loss += loss.item()

  val_loss /= len(test_loader)
  print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

  if val_loss < best_loss:
      best_loss = val_loss
      counter = 0
      # torch.save(model.state_dict(), 'best_model.pth')
  else:
      counter += 1
      if counter >= patience:
          print("Early stopping")
          break

Epoch 1/100, Train Loss: 1.6431, Validation Loss: 1.5811
Epoch 2/100, Train Loss: 1.5347, Validation Loss: 1.4670
Epoch 3/100, Train Loss: 1.4139, Validation Loss: 1.3312
Epoch 4/100, Train Loss: 1.2642, Validation Loss: 1.1543
Epoch 5/100, Train Loss: 1.0713, Validation Loss: 0.9325
Epoch 6/100, Train Loss: 0.8469, Validation Loss: 0.7009
Epoch 7/100, Train Loss: 0.6399, Validation Loss: 0.5137
Epoch 8/100, Train Loss: 0.5267, Validation Loss: 0.3862
Epoch 9/100, Train Loss: 0.3854, Validation Loss: 0.3053
Epoch 10/100, Train Loss: 0.3210, Validation Loss: 0.2512
Epoch 11/100, Train Loss: 0.3274, Validation Loss: 0.2153
Epoch 12/100, Train Loss: 0.2488, Validation Loss: 0.1910
Epoch 13/100, Train Loss: 0.2280, Validation Loss: 0.1725
Epoch 14/100, Train Loss: 0.2122, Validation Loss: 0.1583
Epoch 15/100, Train Loss: 0.2001, Validation Loss: 0.1469
Epoch 16/100, Train Loss: 0.1902, Validation Loss: 0.1378
Epoch 17/100, Train Loss: 0.1822, Validation Loss: 0.1303
Epoch 18/100, Train Los

# 5. Evaluating Model

In [85]:
model.eval()

test_loss, correct = .0, 0

with torch.no_grad():
  for data, target in test_loader:
    data, target = data.permute(1, 0, 2).to(device, dtype = torch.float32), target.to(device)

    output = model(data)

    test_loss += criterion(output, target).item()

    correct += (output.argmax(1) == target).sum().item()

print('Test Loss: {:.6f}'.format(test_loss / len(test_loader)))
print('Accuracy: {:.3f}%'.format(100. * correct / len(test_loader.dataset)))


Test Loss: 0.045890
Accuracy: 98.630%


# 6. Downloading Model

In [90]:
result = torch.jit.trace(model, data.permute(1, 0, 2).to(device, dtype = torch.float32))

result.save(f"GRU_{INPUT_SIZE}I_{HIDDEN_SIZE}H_{NUM_LAYERS}L_{WINDOW_SIZE}W.pt")